In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
num_epochs = 10
num_classes = 10
batch_size = 128

# Size for each sequence and number of seeq
chunk_size = 28
num_chunks = 28
rnn_size = 128

# Init
X = tf.placeholder('float', [None, num_chunks, chunk_size])
y = tf.placeholder('float')

In [9]:
def RNN(X):
    hy = {'weights': tf.Variable(tf.random_normal([rnn_size, num_classes])),
           'biases': tf.Variable(tf.random_normal([num_classes]))}
    X = tf.transpose(X, [1, 0, 2])
    X = tf.reshape(X, [-1, chunk_size])
    X = tf.split(X, num_chunks, 0)
    
    lstm_cell = rnn.BasicLSTMCell(rnn_size) 
    outputs, states = rnn.static_rnn(lstm_cell, X, dtype=tf.float32)
    
    # outputs[-1] is the most recent hidden state
    output = tf.matmul(outputs[-1],hy['weights'] + hy['biases'])
    
    return output

In [16]:
def train_RNN(X):
    pred = RNN(X)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(num_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_X, epoch_y = mnist.train.next_batch(batch_size)
                epoch_X = epoch_X.reshape((batch_size, num_chunks, chunk_size))
                _, c = sess.run([optimizer, loss], feed_dict={X: epoch_X, y: epoch_y })
                epoch_loss += c
                
            print("{} epochs completed out of {}".format(epoch+1, num_epochs))
            correct = tf.equal(tf.argmax(pred, 1), tf.argmax(y,1))
            
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
            
            print('Accuracy: ',accuracy.eval({X:mnist.test.images.reshape((-1, num_chunks, chunk_size)), y:mnist.test.labels}))

In [17]:
tf.reset_default_graph()
# Init
X = tf.placeholder('float', [None, num_chunks, chunk_size])
y = tf.placeholder('float')
train_RNN(X)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 epochs completed out of 10
Accuracy:  0.9498
1 epochs completed out of 10
Accuracy:  0.9645
2 epochs completed out of 10
Accuracy:  0.977
3 epochs completed out of 10
Accuracy:  0.9801
4 epochs completed out of 10
Accuracy:  0.981
5 epochs completed out of 10
Accuracy:  0.9825
6 epochs completed out of 10
Accuracy:  0.9816
7 epochs completed out of 10
Accuracy:  0.983
8 epochs completed out of 10
Accuracy:  0.9853
9 epochs completed out of 10
Accuracy:  0.9847
